In [ ]:
%%writefile submission.py

from typing import Tuple, Optional

import numpy as np
import pandas as pd


class MyConfig:
    P_NOISY_DECISION = 0.1

assert isinstance(MyConfig.P_NOISY_DECISION, float)
assert 0.00 <= MyConfig.P_NOISY_DECISION <= 1.00

class HandSigns:
    ROCK: int = 0
    PAPER: int = 1
    SCISSORS: int = 2
    ALL: Tuple[int] = (ROCK, PAPER, SCISSORS)

    @classmethod
    def predetor_of(cls, sign: int) -> int:
        """Return predetor of given sing.
        """
        assert(sign in cls.ALL)
        return (sign + 1) % 3
    
    @classmethod
    def win(cls, my_action: int, opponent_action: int) -> bool:
        return my_action == cls.predetor_of(opponent_action)


class History:

    def __init__(self):
        
        self.total = [[0, 0, 0], [0, 0, 0]]
        self.log = []
        self.last_agent_action = None
        self.last_opponent_action = None
        self.note = ''


    def memorize(self,
                 agent_action: Optional[int] = None,
                 last_opponent_action: Optional[int] = None,
                 note: Optional[str] = None) -> None:
        """Save agent's action done on this step as the last action.
        """
        if agent_action is not None:
            self.last_agent_action = agent_action
        
        if last_opponent_action is not None:
            self.last_opponent_action = last_opponent_action
        
        if note is not None:
            self.note = note
        return None


    def update(self) -> None:
        """Update game history by step.
        
        When it is an first step, save oppenent's last action only.
        If not, new action log is created by last action of agent and opponent.
        """

        # self.last_opponent_action = last_opponent_action
        on_first_step = self.last_opponent_action is None

        if not on_first_step:  # not first step
            # create new log
            self.log.append((self.last_agent_action,
                             self.last_opponent_action,
                             HandSigns.win(self.last_agent_action, self.last_opponent_action),
                             self.note))
            self.total[0][self.last_agent_action] += 1
            self.total[1][self.last_opponent_action] += 1
        return None


#     def write_log(filepath: str = 'history.csv') -> None:
#         df = pd.DataFrame(history.log, columns=['agent', 'opponent', 'win', 'note'])
#         df.to_csv(filepath)
#         return None


    def most_frequent_actions(self, idx: int = 1) -> list:
        '''Return action taken most frequently by player.
        
        Parameters
        ----------
        idx: int, default = 1.
            0 is my agent, 1 is opponent.
        '''
        assert(idx in (0, 1))  # 0: agent, 1: opponent
        total = self.total[idx]
        max_freq = max(total)
        most_frequent_actions = []
        for sign in HandSigns.ALL:
            n_freq = total[sign]
            if n_freq == max_freq:
                most_frequent_actions.append(sign)
        return most_frequent_actions

    
history = History()
def agent(observation, configuration):        
    global history
    last_opponent_action = None if observation.step < 1 else observation.lastOpponentAction
    history.memorize(last_opponent_action=last_opponent_action)
    history.update()

    my_action = HandSigns.ROCK
    note = ''
    if np.random.choice([True, False], p=[MyConfig.P_NOISY_DECISION, 1 - MyConfig.P_NOISY_DECISION]):
        my_action = np.random.randint(3)
        note = 'Noise'
    else:
        # Decide my action with assuming that opponent will take an action 
        # that opponent have taken most freqently.
        # If all hand-signs have same frequency, my action is chosen randomly.
        opponent_most_frequent_actions = history.most_frequent_actions(1)
        if len(opponent_most_frequent_actions) == 3:
            # All hand-signs have same frequency.
            my_action = np.random.randint(3)
            note = 'Random choice'
        elif len(opponent_most_frequent_actions) == 2:
            # 2 hand-sings have same frequency.
            if opponent_most_frequent_actions == {HandSigns.ROCK, HandSigns.PAPER}:
                my_action = HandSigns.PAPER
            elif opponent_most_frequent_actions == {HandSigns.ROCK, HandSigns.SCISSORS}:
                my_action = HandSigns.ROCK
            else:
                my_action = HandSigns.SCISSORS
            note = "Counter against most frequent action"
        else:
            most_frequent_action = opponent_most_frequent_actions[0]
            my_action = HandSigns.predetor_of(most_frequent_action)
            note = "Counter against most frequent action"

    history.memorize(agent_action=my_action, note=note)
    return my_action

# Evaluation

Creadit:  
https://www.kaggle.com/taahakhan/rps-agent-pool-evaluation

In [ ]:
from kaggle_environments.envs.rps.agents import *
from kaggle_environments import make

def randomAgent(obs, config):
    return random.randint(0, 2)

env = make('rps', debug = True)

# Pass in your agent to evaluate
def evaluate(agent):
    
    pool = [
        rock,
        paper,
        scissors,
        copy_opponent,
        reactionary,
        counter_reactionary,
        statistical,
        randomAgent,
    ]


    print()
    overall = 0

    for opponent in pool:

        env.reset()

        env.run([agent, opponent])
        json = env.toJSON()
        rewards = json['rewards']

        opponentName = str(opponent).split()[1].capitalize()

        padding = ''; length = len(str(rewards))
        if length < 15: padding = ' ' * (15 - length)

        print(f'Rewards: {rewards}{padding}  Agent vs {opponentName}')
        overall += rewards[0]
    
    overall /= len(pool)
    print(f'Overall Score: {overall}')
    
    return overall

In [ ]:
score = evaluate('/kaggle/working/submission.py')
if score <= 0:
    print('\nU MIGHT WANNA FIX SOME THINGS BUDDY')
else:
    print('\nITS GREAT! NOW CHUCK IT INTO THE LEADERBOARD')

In [ ]:
env.reset()
env.run(['submission.py', reactionary])
env.render(mode="ipython", width=500, height=500)

In [ ]:
env.reset()
env.run(['submission.py', statistical])
env.render(mode="ipython", width=500, height=500)